In [79]:
!pip install optuna -q

In [80]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer

import optuna

In [81]:

X_test = pd.read_csv('../data/Xte.csv',sep=',',index_col=0).values
X_train = pd.read_csv('../data/Xtr.csv',sep=',',index_col=0).values

y = pd.read_csv('../data/Ytr.csv',sep=',',index_col=0).values

In [83]:
X_train.shape,X_test.shape

((2000, 1), (1000, 1))

In [84]:
def getKmers(sequence, size):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]
def get_n_grams(data1,data2,n):
    X_te = []
    X = []


    cv = CountVectorizer()
    for i in data1:
        sentence = ' '.join(getKmers(i[0], size=n))
        X.append(sentence)

    for i in data2:
        sentence = ' '.join(getKmers(i[0], size=n))
        X_te.append(sentence)

    
    X_later = X+X_te
    X = cv.fit_transform(X+X_te).toarray()
    
    return X
# breakS
    

In [86]:
X_2 = get_n_grams(X_train,X_test,3)
X_4 = get_n_grams(X_train,X_test,4)

X_6[:15,:15],X_4[:15,:15]

(array([[ 1,  5,  8,  1,  9, 12,  1, 10,  5, 13, 19,  2,  0,  2, 11],
        [ 2,  3, 10,  2,  7,  5,  1,  7,  5,  9, 23,  8,  3,  2, 12],
        [ 3,  6,  4,  3,  4, 16,  8,  9,  8,  7,  9,  5,  1,  8,  7],
        [ 4,  4, 10,  0,  7,  8,  4,  6,  7, 10, 20,  4,  1,  3,  6],
        [ 4, 12,  4,  3,  9,  9,  2, 10,  1,  5,  4, 10,  9,  5,  9],
        [ 4,  6,  6,  0,  6, 16, 10, 10,  4, 13,  7,  4,  2,  6,  5],
        [ 4,  2,  9,  2,  4,  5,  0, 11,  4,  9, 11, 10,  5,  4, 15],
        [ 7,  4, 10,  5, 10, 13,  0,  9,  6,  8,  8,  2,  3,  6,  6],
        [ 1,  1, 12,  0,  3, 18, 16,  3,  9, 18,  7,  4,  1,  3,  2],
        [ 3,  4,  7,  5,  6, 13,  4,  7,  6,  7,  8,  9,  4,  6, 11],
        [ 9,  4, 18,  2, 13,  7,  0,  6,  7, 10,  6,  4,  3,  6,  3],
        [ 3,  5,  6,  4,  7,  8,  3, 15,  4, 11,  5,  3,  4, 10,  9],
        [ 8,  6, 12,  3,  8,  2,  2,  7,  9,  5,  7,  7,  4,  6,  7],
        [ 8,  4, 11,  4,  7, 12,  0, 10,  9,  9,  7,  2,  2,  4,  9],
        [24,  9,  8,

In [87]:
print('x_train: {} y_train {}'.format(X_2[:2000,:].shape,y.shape))

# print('x_train: {} y_train {}'.format(X_train.shape,y.shape))
# print('x_test: {}'.format(X_te.shape))

x_train: (2000, 64) y_train (2000, 1)


# Kernel Test

In [88]:
X_tr_2= scale(X_2[:2000,:])
X_te_2 = scale(X_2[2000:,:])

X_tr_4= scale(X_4[:2000,:])
X_te_4 = scale(X_4[2000:,:])

In [89]:
def string_kernel(X, Y):
    R = np.zeros((len(X), len(Y)))
    for i in range(len(X)):
        for j in range(len(Y)):
            R[i, j] = (np.dot(X[i],Y[j])+1)**3
    return R

In [91]:
X_tr = string_kernel(X_tr_2,X_tr_2)

In [92]:


X_train, X_val, y_train, y_val = train_test_split(
    X_tr, y, test_size=0.33, random_state=42)

print(X_train.shape,X_val.shape,y_train.shape, y_val.shape,X_test.shape)

(1340, 2000) (660, 2000) (1340, 1) (660, 1) (1000, 1)


### Model Test

In [93]:
class logisticregression():
    def __init__(self,train_data,train_labels,lamda=0.2,lr=0.01,decay=10,batch_size=None,epoch=10,print_every = 10):
        dummy_once = np.ones((len(train_data),1))
        self.train_data = np.hstack((dummy_once,train_data))
        self.train_labels = train_labels
        
        self.params = np.zeros((len(self.train_data[0]),1))
        
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        self.print_every = print_every
        self._lambda = lamda
        self.decay = decay
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def cost(self,y,y_pred):
        return -np.mean(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))
    
    def gradient(self,y,y_pred,x):
        return np.dot(x.T,(y_pred-y))+(2*self._lambda*self.params)
    
    def train(self):
        for i in range(self.epoch):
            y_pred = self.sigmoid(np.dot(self.train_data,self.params))
            loss = self.cost(self.train_labels,y_pred)
            
            gra = self.gradient(self.train_labels,y_pred,self.train_data)
            
            self.params -= self.lr*gra
            
            self.lr *= (1. / (1. + self.decay * i))
            
#             if self.print_every:
#                 if i%self.print_every == 0 or i == self.epoch-1:
#                     print('Epoch : {}  Loss: {}'.format(i,loss))
    def predict(self,test_data):
        result = self.sigmoid(np.dot(test_data,self.params[1:])+self.params[0])
        result[result > 0.5 ] = 1
        result[result <= 0.5 ] = 0
        return result
    
    def evaluate(self,test_data,labels):
        accuracy = accuracy_score(self.predict(test_data),labels)
        return accuracy

In [94]:
def cross_validate(x_data,y_data,lr,lamda=0.2,epoch=10,k=5,decay=10):
    if len(x_data)%k != 0:
        print('cant vsplit',len(x_data),' by ',k)
        return
    
    x_data_splitted = np.vsplit(x_data,k)
    y_data_splitted = np.vsplit(y_data,k)
    
    aggrigate_result = []
    for i in range(len(x_data_splitted)):
        train = []
        test = []
        items = [j for j in range(len(x_data_splitted)) if j !=i ]
        x_test = x_data_splitted[i]
        y_test = y_data_splitted[i]
        for item in items:
            if len(train) == 0:
                x_train = x_data_splitted[item]
                y_train = y_data_splitted[item]
            else:
                x_train = np.concatenate((x_train,x_data_splitted[item]), axis=0)
                y_train = np.concatenate((y_train,y_data_splitted[item]), axis=0)
            
        logistic = logisticregression(x_train,y_train,lamda=lamda,lr=lr,decay=decay,epoch=epoch)
        logistic.train()
        
        result = logistic.evaluate(x_test,y_test)
        aggrigate_result.append(result)
        
        value = sum(aggrigate_result)/len(aggrigate_result)
    return value if value!= None else 0

In [95]:
def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    lamda = trial.suggest_loguniform('lamda', 0.01, 0.5)
    k =  trial.suggest_categorical('k', [4,5,8,10])
    epoch =  trial.suggest_int('epoch', 10, 20)
    decay = trial.suggest_int('decay', 3, 10)
    return cross_validate(X_tr, y,lr=lr,lamda=lamda,k=k,epoch=epoch,decay=decay)

In [96]:
# cross_validate(X_train_mat100, y,0.001,10)

import optuna

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective, n_trials=200,show_progress_bar=True)

/usr/local/lib/python3.6/dist-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning:

overflow encountered in exp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning:

invalid value encountered in multiply



[I 2020-05-26 13:45:55,061] Finished trial#0 with value: 0.5145 with parameters: {'lr': 6.235579877986309e-05, 'lamda': 0.14173412997138396, 'k': 5, 'epoch': 18, 'decay': 7}. Best is trial#0 with value: 0.5145.
[I 2020-05-26 13:45:55,194] Finished trial#1 with value: 0.5129999999999999 with parameters: {'lr': 0.014719889984033585, 'lamda': 0.03005429962230471, 'k': 5, 'epoch': 11, 'decay': 9}. Best is trial#0 with value: 0.5145.
[I 2020-05-26 13:45:55,358] Finished trial#2 with value: 0.5135 with parameters: {'lr': 0.04190786950601587, 'lamda': 0.038920337513285136, 'k': 10, 'epoch': 18, 'decay': 9}. Best is trial#0 with value: 0.5145.
[I 2020-05-26 13:45:55,508] Finished trial#3 with value: 0.516 with parameters: {'lr': 0.04387466361069148, 'lamda': 0.010747135677537942, 'k': 8, 'epoch': 16, 'decay': 6}. Best is trial#3 with value: 0.516.
[I 2020-05-26 13:45:55,700] Finished trial#4 with value: 0.515 with parameters: {'lr': 0.00047091319687875364, 'lamda': 0.04072697236464424, 'k': 8,

In [312]:
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)

df.sort_values(by=['value'])

,number,value,duration,params_decay,params_epoch,params_k,params_lamda,params_lr
0,0,0.6045,00:00:00.235365,8,19,10,0.368972,0.016493
29,29,0.6045,00:00:00.218716,7,17,10,0.089794,0.009743
8,8,0.6065,00:00:00.233999,10,19,10,0.013069,0.000233
179,179,0.6075,00:00:00.191209,8,10,10,0.034966,0.001303
57,57,0.6085,00:00:00.202923,6,11,10,0.019213,0.001156
...,...,...,...,...,...,...,...,...
97,97,0.6295,00:00:00.189959,6,10,4,0.022685,0.001447
95,95,0.6295,00:00:00.191692,6,10,4,0.040199,0.001438
122,122,0.6295,00:00:00.185681,6,10,4,0.024073,0.001434
83,83,0.6295,00:00:00.204314,7,11,4,0.010491,0.001443


In [314]:
cross_validate(X_tr,y,lamda=0.050740,epoch=10,lr=0.050740,k=4,decay=7)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning:

invalid value encountered in multiply



0.623

In [343]:
# X_train, X_test, y_train, y_test
# logistic = logisticregression(X_train,y_train,lamda=0.362124,epoch=15,print_every=1,lr=0.000254)
#
logistic = logisticregression(X_train,y_train,lamda=0.05,epoch=10,lr=0.001383,decay=7,print_every=1)
logistic.train()
        
print(logistic.evaluate(X_train,y_train))
print(logistic.evaluate(X_val,y_val))

Epoch : 0  Loss: 0.6931471805599453
Epoch : 1  Loss: 0.6614940106899204
Epoch : 2  Loss: 0.6576564995558486
Epoch : 3  Loss: 0.6530411387034815
Epoch : 4  Loss: 0.6529215147131578
Epoch : 5  Loss: 0.6529162816687935
Epoch : 6  Loss: 0.6529161015244107
Epoch : 7  Loss: 0.6529160965206913
Epoch : 8  Loss: 0.652916096404326
Epoch : 9  Loss: 0.6529160964019987
0.6313432835820896
0.55


In [317]:

X_test_mat100 = scale(X_te)

In [325]:

sumbission = []
for i in range(len(X_test_mat100)):
    result = logistic.predict(X_test_mat100[i])
    sumbission.append([i,int(result)])
    result

In [326]:
# sumbission
df = pd.DataFrame(sumbission)
df.columns = ['Id','Bound']
df.to_csv('test_64_cross_validated.csv',index=False)

In [327]:
df.head(50)

,Id,Bound
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0
5,5,1
6,6,1
7,7,1
8,8,1
9,9,0


# SVM Test


In [383]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’
# clf = make_pipeline(StandardScaler(), SVC(kernel='poly'))
# clf.fit(X_train,y_train)
# clf.score(X_val,y_val)

In [385]:

X_later.shape

AttributeError: 'list' object has no attribute 'shape'

In [388]:
X = np.array(X_later)
X.shape

(3000,)

In [390]:
X_tr= X[:2000]
X_te = X[2000:]

X_train, X_val, y_train, y_val = train_test_split(
    X_tr, y, test_size=0.33, random_state=42)

print(X_train.shape,X_val.shape,y_train.shape, y_val.shape,X_test.shape)

(1340,) (660,) (1340, 1) (660, 1) (1000, 1)


In [391]:
X_train

array(['gg gg gt tc ct tg gc cc cc cc cc ca ac ca ag gc cc cc cc ct ta ag gc ct tg gt tt tc cc cg gt tc ct tg gt tt ta at ta ag ga at tt tg gc ca aa ac cc cc cc ct tg gt tg gt tg gg ga aa at tg gc ca ag gt tg gt tc cc ct tt tt tt tc cc cc cc ct ta ag gg gg ga ag gt tg gg gg gt tg gt tg gc ca at tg ga ac ct',
       'at tg gt tg gt tg gt ta at tt tt ta aa aa aa aa at ta aa aa ac ct tt tt ta ag gt tt tt tt tt tc ct tc cc ct tc ca at ta at tg ga ag gc ct ta ac ca at tc ca at tt tt tt tc cc ct tg gt tt ta ac ca at ta ag ga aa at tt ta aa at ta aa ac cc ct tg ga aa ac ct ta ag ga ag gt tc cc ct ta at tc ct tc ct tg',
       'cg gg gc cc cc ct tt tg gg gg gt ta ag gg ga ac ca ag gg gt tt tt tg gc ct tt tt tg gt ta aa at tg gt tc ct tt tc cc ct tc cc cc ca ac ct tc cc ca ac cc cc cg gc cc ct tc cc cc cc ca ag ga ag gg gt ta ac cc cc ct tt ta ac cc ca ag gt tg gt tc ct tc cc ct tg gg ga aa ac cc ct tt tc cc ca ag ga ag ga',
       ...,
       'ct tc cc ct tc cc cc ca ac ct tt tc ct tt tc cc ct

In [394]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer


clf = Pipeline([('vect', CountVectorizer()),
                             ('tfidf', TfidfTransformer()),
                             ('clf-svm', LogisticRegression(penalty = 'l2'))])
clf.fit(X_train,y_train)
clf.score(X_val,y_val)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



0.5424242424242425